# very easy plot on italian corona virus cases devided by provinces 

### importing packages

In [81]:
from datetime import datetime
from datetime import timedelta
from datetime import date
import pandas as pd
import numpy as np
import plotly as py 
import plotly.graph_objs as go
import ipywidgets as widgets

### setting function that upload avaiable data up to the current day from https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/, keeping just the new positives values 

In [237]:
def load_provinces_data():
    main_url = 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/'
    url_provinces = main_url + 'dati-province/dpc-covid19-ita-province-{}.csv'.format('20200224')
    provinces = pd.read_csv(url_provinces, error_bad_lines=False)
    provinces = provinces.rename(columns={'totale_casi' : 'casi_giorno_0'})
    dt = datetime(2020, 2, 25)
    end = datetime.combine(date.today(), datetime.min.time())
    step = timedelta(days=1)
    result = []
    number = 1
    while dt < end:
        result.append(dt.strftime('%Y%m%d'))
        dt += step
    for i in result:
      url_provinces = main_url + 'dati-province/dpc-covid19-ita-province-{}.csv'.format(i)
      province = pd.read_csv(url_provinces, error_bad_lines=False)
      provinces = pd.concat([provinces,province.totale_casi],axis=1)
      provinces = provinces.rename(columns={'totale_casi' : 'casi_giorno_{}'.format(number)})
      number = number + 1
    provinces= provinces.drop(['note_it', 'note_en','data','stato','codice_provincia'], axis=1)
    #provinces = provinces.dropna(axis=0)
    return provinces


In [238]:
data = load_provinces_data()
data = data.dropna(axis=0)
data = data.reset_index(drop=True)
data

,codice_regione,denominazione_regione,denominazione_provincia,sigla_provincia,lat,long,casi_giorno_0,casi_giorno_1,casi_giorno_2,casi_giorno_3,...,casi_giorno_102,casi_giorno_103,casi_giorno_104,casi_giorno_105,casi_giorno_106,casi_giorno_107,casi_giorno_108,casi_giorno_109,casi_giorno_110,casi_giorno_111
0,13,Abruzzo,Chieti,CH,42.351032,14.167546,0,0,0,0,...,824,825,825,825,825,825,825,826,828,827
1,13,Abruzzo,L'Aquila,AQ,42.351222,13.398438,0,0,0,0,...,247,247,247,247,247,246,246,246,246,247
2,13,Abruzzo,Pescara,PE,42.464584,14.213648,0,0,0,0,...,1527,1524,1531,1531,1532,1533,1534,1536,1538,1539
3,13,Abruzzo,Teramo,TE,42.658918,13.704400,0,0,0,1,...,659,662,662,662,662,662,662,663,663,666
4,17,Basilicata,Matera,MT,40.667512,16.597924,0,0,0,0,...,208,208,208,208,209,209,210,210,210,210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,5,Veneto,Rovigo,RO,45.071073,11.790070,0,0,0,0,...,442,442,442,443,443,443,443,442,443,443
102,5,Veneto,Treviso,TV,45.667546,12.245074,0,1,2,22,...,2661,2663,2663,2665,2665,2665,2666,2665,2666,2666
103,5,Veneto,Venezia,VE,45.434905,12.338452,0,7,8,14,...,2666,2667,2668,2668,2668,2671,2673,2675,2674,2678
104,5,Veneto,Verona,VR,45.438390,10.993527,0,0,0,0,...,5103,5106,5105,5105,5106,5107,5107,5109,5110,5110


In [239]:
dt = datetime(2020, 2, 25)
end = datetime.combine(date.today(), datetime.min.time())
delta =  (end - dt).days

### setting the plot using plotly

In [246]:
layout = go.Layout(
    title= 'PROVINCES',
    yaxis=dict(
        title= 'positives',
    ),
    xaxis=dict(
        title= 'days'
    )
)

def update_plot(signals):
    datax = []
    for i in signals:
        trace1 = go.Scatter(
            x = np.arange(0,delta),
            y = data.iloc[i,6:delta+6],
            name = data.denominazione_provincia[i]
        )
        datax.append(trace1)
    
    fig = go.Figure(data=datax,layout=layout)
    py.offline.iplot(fig)
    
signals= widgets.SelectMultiple(options=range((106)), value=[0,], description = 'privince')
widgets.interactive(update_plot,signals=signals)

interactive(children=(SelectMultiple(description='privince', index=(0,), options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9…